# 이게 크롤링 하다가 지금 매매랑 전세 월세의 코드가 다르다.
# 이거 찾아는 보겠는데 혹시 ios 단에서 테스트 할 때 값 이상하면 말좀 해달라.
# 이거 매매 종류 볼때 매매랑 전세/월세 다르면 코드 다르게 돌리자
# 이거 지금 성수에 매매가 5개인데 하나만 달라서 이거만 따로 빼면 될듯

In [1]:
from selenium import webdriver
dabang_url ='https://www.dabangapp.com/search#/map?filters=%7B%22multi_room_type%22%3A%5B2%2C1%2C0%5D%2C%22selling_type%22%3A%5B0%2C1%2C2%5D%2C%22deposit_range%22%3A%5B0%2C999999%5D%2C%22price_range%22%3A%5B0%2C999999%5D%2C%22trade_range%22%3A%5B0%2C999999%5D%2C%22maintenance_cost_range%22%3A%5B0%2C999999%5D%2C%22include_maintenance_option1%22%3Atrue%2C%22room_size%22%3A%5B0%2C999999%5D%2C%22supply_space_range%22%3A%5B0%2C999999%5D%2C%22room_floor_multi%22%3A%5B1%2C2%2C3%2C4%2C7%2C6%2C5%5D%2C%22division%22%3Afalse%2C%22duplex%22%3Afalse%2C%22room_type%22%3A%5B1%2C2%5D%2C%22enter_date_range%22%3A%5B0%2C999999%5D%2C%22parking_average_range%22%3A%5B0%2C999999%5D%2C%22household_num_range%22%3A%5B0%2C999999%5D%2C%22parking%22%3Afalse%2C%22animal%22%3Afalse%2C%22short_lease%22%3Afalse%2C%22full_option%22%3Afalse%2C%22built_in%22%3Afalse%2C%22elevator%22%3Afalse%2C%22balcony%22%3Afalse%2C%22loan%22%3Afalse%2C%22safety%22%3Afalse%2C%22pano%22%3Afalse%2C%22deal_type%22%3A%5B0%2C1%5D%7D&position=%7B%22location%22%3A%5B%5B127.03782637531295%2C37.518247186167436%5D%2C%5B127.07787253873349%2C37.56435875152705%5D%5D%2C%22center%22%3A%5B127.05784328891325%2C37.541304716670695%5D%2C%22zoom%22%3A6%7D&search=%7B%22id%22%3A%2211200114%22%2C%22type%22%3A%22region%22%2C%22name%22%3A%22%EC%84%B1%EC%88%98%EB%8F%991%EA%B0%80%22%7D&tab=all'

## start staticmethod 
    - SocialLogin.start()
    - SalesForm.start()

In [2]:
driver = webdriver.Chrome('/Users/mac/projects/ChromeWebDriver/chromedriver')
driver.implicitly_wait(3)

In [3]:
driver.get(dabang_url)
seongsoo_1dong_detail_list = driver.find_elements_by_xpath("/html/body/div[1]/div/div[2]/div[1]/div/div[2]/ul/li/div/a")

In [4]:
seongsoo_1dong_detail_list[0].get_attribute("href")

'https://www.dabangapp.com/room/5e6f0321aeeb591276221de3'

In [5]:
bang_url_list = []

In [6]:
for url in seongsoo_1dong_detail_list:
    bang_url_list.append(url.get_attribute('href'))

In [7]:
# [driver.get(url) for url in bang_url_list]

# test code start

In [8]:
from selenium.common.exceptions import NoSuchElementException

In [9]:
driver.get(bang_url_list[0])

In [15]:
driver.implicitly_wait(3) 
# 이거 안하면 아직 페이지 로딩 안되서 에러남 버튼 아직 없음

In [14]:
try:
    driver.find_element_by_xpath("/html/body/div[1]/div/div[4]/div/div/button").click()
except NoSuchElementException:
    pass    

WebDriverException: Message: unknown error: Element <button class="styled__ShowBtn-sc-1xkewlx-3 jQYbpN">...</button> is not clickable at point (638, 598). Other element would receive the click: <html>...</html>
  (Session info: chrome=80.0.3987.162)
  (Driver info: chromedriver=2.44.609545 (c2f88692e98ce7233d2df7c724465ecacfe74df5),platform=Mac OS X 10.14.4 x86_64)


In [16]:
unrefined_description = driver.find_elements_by_xpath("/html/body/div[1]/div/div[4]/div/div")

In [17]:
description = unrefined_description[0].get_attribute("innerText")

In [18]:
description

'빌트인 수납공간 풍부,풀옵션,남향,살기 좋은 오피스텔\n**성수동 리버하우스**\n\n*2004년 1월 준공,320세대\n*2호선 뚝섬역 도보 5분 거리\n*지하 및 지상주차장 완비!\n*복층 제외 바닥 전용면적만 무려 12평!\n*방1,거실,주방,복층\n*잘 짜여진 빌트인-옷장, 각종 수납장, 드럼세탁기, 가스레인지, 에어컨 \n*실제로 보면 더 반하는 오피스텔!\n\n\n\n**성수동 가나부동산** \n정직과 신뢰를 바탕으로 20년간 성수동 뚝섬역에서 중개업을 하고 있으며, 많은 경험과 데이터로 다양한 매물을 확보하고 있습니다. 궁금하신 사항은 언제든 연락주세요~^^\n♣주소-성수동 중앙하이츠 상가 108호 (뚝섬역 3번 출구 도보1분, 파리바게트 바로 옆 건물)\n\n\n\n\n\n접기'

In [294]:
description.replace("\n", "")

'빌트인 수납공간 풍부,풀옵션,남향,살기 좋은 오피스텔**성수동 리버하우스***2004년 1월 준공,320세대*2호선 뚝섬역 도보 5분 거리*지하 및 지상주차장 완비!*복층 제외 바닥 전용면적만 무려 12평!*방1,거실,주방,복층*잘 짜여진 빌트인-옷장, 각종 수납장, 드럼세탁기, 가스레인지, 에어컨 *실제로 보면 더 반하는 오피스텔!**성수동 가나부동산** 정직과 신뢰를 바탕으로 20년간 성수동 뚝섬역에서 중개업을 하고 있으며, 많은 경험과 데이터로 다양한 매물을 확보하고 있습니다. 궁금하신 사항은 언제든 연락주세요~^^♣주소-성수동 중앙하이츠 상가 108호 (뚝섬역 3번 출구 도보1분, 파리바게트 바로 옆 건물)접기'

# 이건 데이터 어떻게 념겨줄건지 이야기 할 때 내가 안바꾸고 거기서 바꾸는게 더 편하지 않겠냐 물어보기 -- 위에 출력한 텍스트 두개

In [415]:
unrefined_address = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div/div/p')
address = unrefined_address[0].get_attribute("innerText")
# 이건 테이블 분리 안하고 그냥 여기에 넣으면 될 듯 동 단위여서 의미가 없음
# 테이블 수정함 PostAddress
address

'서울특별시 성동구 성수동1가'

In [416]:
unrefined_salesform = driver.find_elements_by_xpath('/html/body/div[1]/div/div[1]/div/ul/li[1]/div')
salesForm = unrefined_salesform[0].get_attribute("innerText")
salesForm

'월세 3000/80\n만원'

In [26]:
salesForm = salesForm.replace('/', ' ')

In [27]:
salesForm = salesForm.replace('\n', '')

In [28]:
salesForm = salesForm.split()

In [29]:
salesForm

['월세', '3000', '80만원']

In [30]:
salesform_type = salesForm[0]

In [31]:
salesform_type

'월세'

In [32]:
salesform_deposit = salesForm[1]

In [33]:
salesform_deposit

'3000'

In [34]:
salesform_monthly = salesForm[2]

In [35]:
salesform_monthly
# SalesForm model의 salePrice라는 속성 지우기, 매매 가격도 deposit으로 통일 하는게 나을듯
# https://www.dabangapp.com/room/5dea1592bfd6ae3b8ca406ce 처럼 월세가 보증금마다 달라지는건 어떻게 할지 고민중

'80만원'

In [36]:
unrefined_floor = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[1]/div')

In [37]:
total_floor = unrefined_floor[0].get_attribute('innerText')

In [38]:
total_floor = total_floor.split('/')

In [39]:
floor = total_floor[0]

In [40]:
floor

'7층 '

In [41]:
totalFloor = total_floor[1]

In [69]:
totalFloor =totalFloor.replace(' ', '')

In [417]:
totalFloor

'13층'

In [43]:
unrefined_area = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[2]/div/span')

In [48]:
area = unrefined_area[0].get_attribute('innerText')

In [50]:
area

'12 / 20평'

In [51]:
driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[2]/div/button').click()

In [52]:
unrefined_area = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[2]/div/span')

In [53]:
supplyArea = unrefined_area[0].get_attribute('innerText')

In [54]:
supplyArea

'39 / 65㎡'

In [227]:
unrefined_shortRent = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[2]/div/table/tbody/tr/td[5]/p')

In [228]:
shortRent = unrefined_shortRent[0].get_attribute('innerText')

In [229]:
shortRent

'불가능'

In [230]:
# if shortRent=='불가능':
#     shortRent= False
# else:
#     shortRent= True

In [275]:
unrefined_management = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[2]/div/table/tbody/tr/td[3]')

In [276]:
unrefined_management = unrefined_management[0].get_attribute('innerText')

In [277]:
unrefined_management = unrefined_management.replace('\n', '')

In [278]:
unrefined_management

'11만 원(청소비, 기타, 수도세, 전기세)'

In [279]:
unrefined_management = unrefined_management.replace(' ', '')

In [280]:
unrefined_management
# 이렇게 한번에 할지 아니면 아래처럼 분리할지 

'11만원(청소비,기타,수도세,전기세)'

In [281]:
unrefined_management = unrefined_management.replace('(', ' ')

In [282]:
unrefined_management

'11만원 청소비,기타,수도세,전기세)'

In [283]:
unrefined_management = unrefined_management.replace(')', ' ')

In [284]:
unrefined_management

'11만원 청소비,기타,수도세,전기세 '

In [285]:
unrefined_management = unrefined_management.replace(',', ' ')

In [286]:
unrefined_management = unrefined_management.split(' ')

In [290]:
unrefined_management
# get_or_create()해야 할 듯 .

['11만원', '청소비', '기타', '수도세', '전기세', '']

In [291]:
managementPay = unrefined_management.pop(0)

In [292]:
managementPay

'11만원'

In [295]:
totalPay = managementPay.split('만')

In [299]:
totalPay = totalPay[0]
# 이걸 MTM필드 totalFee로 한다. 속성으로 만들어서 필터링 할 때 조회 가능하게 고민.

In [293]:
unrefined_management

['청소비', '기타', '수도세', '전기세', '']

## 이게 다방이 방 내놓을때에는 관리비 포함 항목들을 오브젝트로 만들어놨는데 
## 필터링할때에는 포함 항목들이 해당되지 않는다.
## 이게 다방은 없는데 우리는 있는 기능으로 만들 수 있긴 있을 것 같다.

In [300]:
unrefined_parking = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[2]/div/table/tbody/tr/td[4]/p')

In [303]:
unrefined_parking[0].get_attribute('innerText')
# 관리비는 따로 필터링 하지 않아서 그냥 텍스트로만 저장

'가능(무료)'

In [315]:
try:
    unrefined_living_expenses = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[2]/div/div/div/label')
    unrefined_living_expenses_detail = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[2]/div/div/div/span')
except NoSuchElementException:
    pass
# 한달 생활비가 있는 글이 있고 없는 글이 있다.

In [316]:
unrefined_living_expenses= unrefined_living_expenses[0].get_attribute('innerText')
living_expenses = unrefined_living_expenses.replace(' ', '')
living_expenses

'91만원+α'

In [318]:
living_expenses_detail = unrefined_living_expenses_detail[0].get_attribute('innerText')
living_expenses_detail

'(월세 + 관리비 + 주차비)'

In [328]:
unrefined_parking = driver.find_elements_by_xpath("/html/body/div[1]/div/div[5]/div[2]/div/table/tbody/tr/td[4]/p")
parking = unrefined_parking[0].get_attribute('innerText')
parking
# model에서 주차 비용 유무는 그냥 ios에서 보일때만 표시하고 django에 들어올때에는 없으면 안받고 있으면 받음.
# 주차비도 필터링에 조건 되는게 없으니까 따로 숫자 빼지는 않겠음.

'가능(무료)'

In [327]:
unrefined_moveIn = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[9]/div')
moveIn = unrefined_moveIn[0].get_attribute('innerText')
moveIn

'날짜 협의'

In [392]:
try:
    unrefined_option = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[3]/div[1]/div')
except NoSuchElementException:
    pass
unrefined_option = unrefined_option[0].get_attribute('innerText')
unrefined_option = unrefined_option.split('\n\n')
option = unrefined_option
option

['에어컨', '세탁기', '책상', '옷장', '신발장', '냉장고', '가스레인지', '전자도어락']

In [397]:
unrefined_heatingType = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[3]/div')
heatingType = unrefined_heatingType[0].get_attribute('innerText')
heatingType

'개별난방'

In [401]:
unrefined_pet = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[6]/div')
pet = unrefined_pet[0].get_attribute('innerText')
pet

'불가능'

In [405]:
unfined_elevator = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[5]/div')
elevator = unfined_elevator[0].get_attribute('innerText')
elevator

'있음'

### 복층, 1.5룸, 주방 분리형 아직 못찾아서 안넣었음

In [408]:
unrefined_builtIn = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[4]/div')
builtIn = unrefined_builtIn[0].get_attribute('innerText')
builtIn

'빌트인 주방'

In [409]:
unrefined_veranda = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[7]/div')
veranda = unrefined_veranda[0].get_attribute('innerText')
veranda

'없음'

In [410]:
unrefined_depositLoan = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[8]/div')
depositLoan = unrefined_depositLoan[0].get_attribute('innerText')
depositLoan

'불가능'

In [414]:
unrefined_securitySafety = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[3]/div[2]/div')
security = unrefined_securitySafety[0].get_attribute('innerText')
security= security.split('\n\n')
security

['비디오폰', '공동현관', '경비원', 'CCTV', '화재경보기']